# Amenities
## Imports

In [ ]:
import geopandas as gpd
import osmnx as ox
import pandas as pd

## Parameters

In [ ]:
assylum_facilities_filepath = "data/processed/assylum-facilities.gpkg"
student_residences_filepath = "data/processed/student-residences.gpkg"
buffer_dist = 10000
epsg = 2056
dst_filepath = "data/processed/amenities.gpkg"
tags_dict = {
    # public-transport-stops": {"public_transport": True},
    # "restaurants-cafes": {"amenity": ["restaurant", "cafe"]},
    # "schools": {"amenity": "school"},
    # "supermarkets": {"shop": "supermarket"},
    # "banks": {"amenity": "bank"},
    # "sports-fitness-centers": {"leisure": ["sports_centre", "fitness_centre"]},
    # "bakeries": {"shop": "bakery"},
    # "kiosks": {"shop": "kiosk"},
    "pharmacies": {"amenity": "pharmacy"},
    # "swimming-pools-beaches": {
    #     "leisure": ["swimming_pool", "beach_resort"],
    #     "natural": "beach",
    # },
    # "libraries": {"amenity": "library"},
    # "hospitals": {"amenity": "hospital"},
    # "cinemas": {"amenity": "cinema"},
    # "theatres": {"amenity": "theatre"},
    # "post-offices": {"amenity": "post_office"},
    # "night-clubs": {"amenity": "nightclub"},
    # "bars": {"amenity": "bar"},
    # "parks": {"leisure": "park"},
    # "playgrounds": {"leisure": "playground"},
}

## Load data

In [ ]:
assylum_facilities_gdf = gpd.read_file(assylum_facilities_filepath)
student_residences_gdf = gpd.read_file(student_residences_filepath)

## Concatenate the geometry columns of the two dataframes into a single geoseries

In [ ]:
locations_gser = pd.concat(
    [assylum_facilities_gdf["geometry"], student_residences_gdf["geometry"]],
    ignore_index=True,
)

## Get amenities from OSM at a given distance buffer from assylum facilities
### Get the extent by adding a buffer around locations and reproject to WGS84

In [ ]:
# use a multipolygon joining all rows of gdf as the extent to query OSM
extent = locations_gser.buffer(buffer_dist).to_crs("EPSG:4326").unary_union

### Query OSM for amenities given the extent and tags

In [ ]:
amenities_gdf = gpd.GeoDataFrame(columns=["category", "geometry"])
for category, tags in tags_dict.items():
    print(f"Getting amenities for {category}...")
    category_gser = ox.geometries_from_polygon(extent, tags=tags)["geometry"].apply(
        lambda x: x.centroid if x.geom_type != "Point" else x
    )
    amenities_gdf = pd.concat(
        [
            amenities_gdf,
            gpd.GeoDataFrame(
                geometry=category_gser,
            )
            .reset_index()
            .drop(columns="element_type")
            # .set_index("osmid")
            .assign(category=category),
        ],
        ignore_index=True,
    )

### Set the osmid as index and reproject

In [ ]:
amenities_gdf["osmid"] = amenities_gdf["osmid"].astype(int)
amenities_gdf = amenities_gdf.set_index("osmid")
amenities_gdf = amenities_gdf.to_crs(epsg=epsg)

### Save amenities to file

In [ ]:
amenities_gdf.to_file(dst_filepath)

In [ ]:
for category, category_gdf in amenities_gdf.groupby("category"):
    category_gdf.to_file(f"data/processed/amenities-{category}.gpkg")